# Neural Network test

In [2]:
import nn
import numpy as np
from numpy.random import uniform, randint
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import numdifftools as nd

In [187]:
np.random.seed(0)

data = pd.DataFrame()

X_LOW = -5
X_HIGH = 5
N_X = 30

N_samples = 500
N_funcs = 3

data['x'] = range(N_samples)
data['x'] = data['x'].apply(lambda x: np.linspace(X_LOW, X_HIGH, N_X))

def sin(x):
    return uniform() * np.sin(uniform() * x + uniform()) + uniform()

def linear(x):
    return uniform() * x + uniform()

def quadratic(x):
    return uniform() * x ** 2 + uniform() * x + uniform()

funcs = [sin, linear, quadratic]

def random_func(x, i):
    return funcs[i](x)

def i_to_array(i):
    array = np.zeros(shape=N_funcs)
    array[i] = 1
    return array

data['i_func'] = np.random.randint(0, N_funcs, size=N_samples)

data['food'] = data[['x', 'i_func']].apply(lambda x: random_func(x.iloc[0], x.iloc[1]), axis=1)
data['expected'] = data['i_func'].apply(i_to_array)

mean = np.mean(np.mean(data['food']))
data['food'] = data['food'].apply(lambda x: x - mean)
data['food'] /= data['food'].apply(lambda x: np.max(abs(x)))


data.head()

,x,i_func,food,expected
0,"[-5.0, -4.655172413793103, -4.310344827586206,...",0,"[-1.0, -0.9921806773604878, -0.977800894914568...","[1.0, 0.0, 0.0]"
1,"[-5.0, -4.655172413793103, -4.310344827586206,...",1,"[-1.0, -0.9622404156987415, -0.924480831397482...","[0.0, 1.0, 0.0]"
2,"[-5.0, -4.655172413793103, -4.310344827586206,...",0,"[-0.7378824780190136, -0.7958682375951018, -0....","[1.0, 0.0, 0.0]"
3,"[-5.0, -4.655172413793103, -4.310344827586206,...",1,"[-1.0, -0.9746552244804404, -0.949310448960880...","[0.0, 1.0, 0.0]"
4,"[-5.0, -4.655172413793103, -4.310344827586206,...",1,"[-1.0, -0.9563278626514361, -0.912655725302872...","[0.0, 1.0, 0.0]"


In [188]:
TEST_SIZE = 0.2
data_train = data.sample(frac=(1-TEST_SIZE), random_state=0)
data_test = data.drop(data_train.index)

In [200]:
%load_ext autoreload
%autoreload 2

N_HIDDEN = 10
ETA = 1.e-3
N_EPOCHS = 100
BATCH_SIZE = 20

ADAPTIVE_ETA = False
ETA_LIMIT = 1.e-6
ETA_FACTOR = 1.5
CONTINUE_MIN_ETA = True

RANDOM_SEED = 5
MSG_FREQ = 10

ACT_FUNCS = [nn.sigmoid, nn.sigmoid]
COST_FUNC = nn.mean_square

network_shape = (N_X, N_HIDDEN, N_funcs)

neural_network = nn.NeuralNetwork(shape=network_shape, act_funcs=ACT_FUNCS, cost_func=COST_FUNC,
                                  eta=ETA, eta_limit=ETA_LIMIT, adaptive_eta=ADAPTIVE_ETA, eta_factor=ETA_FACTOR,
                                  continue_min_eta=CONTINUE_MIN_ETA, random_seed=RANDOM_SEED)

df_stats = neural_network.train(df=data_train[['food', 'expected']], n_epochs=N_EPOCHS, msg_freq=MSG_FREQ, batch_size=BATCH_SIZE)
df_stats.to_csv('stats.csv', mode='a', header=None)
pd.DataFrame(neural_network.parameters).to_csv('parameters.csv')
df_stats

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Starting training.

[2023-04-07 21:08:58.620848]: Currently at epoch 18/100
Average duration per epoch: 0.58 s.
Estimated time of finishing: 2023-04-07 21:09:46.298518


[2023-04-07 21:09:08.646769]: Currently at epoch 33/100
Average duration per epoch: 0.62 s.
Estimated time of finishing: 2023-04-07 21:09:50.251223


[2023-04-07 21:09:18.878075]: Currently at epoch 49/100
Average duration per epoch: 0.63 s.
Estimated time of finishing: 2023-04-07 21:09:50.855126


[2023-04-07 21:09:29.201471]: Currently at epoch 66/100
Average duration per epoch: 0.62 s.
Estimated time of finishing: 2023-04-07 21:09:50.346615


[2023-04-07 21:09:39.604999]: Currently at epoch 83/100
Average duration per epoch: 0.62 s.
Estimated time of finishing: 2023-04-07 21:09:50.142942


[2023-04-07 21:09:49.833531]: Currently at epoch 100/100
Average duration per epoch: 0.62 s.
Estimated time of finishing: 2023-04-07 21:09:49.

,start,stop,E_mean,eta,acc_train,acc_test,est_finish
0,2023-04-07 21:08:48.155026,2023-04-07 21:08:48.721330,0.254772,0.001,0.305,None,None
1,2023-04-07 21:08:48.769340,2023-04-07 21:08:49.375203,0.252167,0.001,0.305,None,None
2,2023-04-07 21:08:49.418224,2023-04-07 21:08:50.064621,0.243529,0.001,0.305,None,None
3,2023-04-07 21:08:50.106648,2023-04-07 21:08:50.671779,0.251674,0.001,0.305,None,None
4,2023-04-07 21:08:50.714786,2023-04-07 21:08:51.281184,0.235795,0.001,0.285,None,None
...,...,...,...,...,...,...,...
95,2023-04-07 21:09:47.462995,2023-04-07 21:09:48.017105,0.210942,0.001,0.3825,None,2023-04-07 21:09:50.142942
96,2023-04-07 21:09:48.059114,2023-04-07 21:09:48.615474,0.212693,0.001,0.43,None,2023-04-07 21:09:50.142942
97,2023-04-07 21:09:48.658495,2023-04-07 21:09:49.203445,0.210327,0.001,0.3925,None,2023-04-07 21:09:50.142942
98,2023-04-07 21:09:49.246442,2023-04-07 21:09:49.790533,0.211048,0.001,0.385,None,2023-04-07 21:09:50.142942


In [201]:
fig = go.Figure([
    go.Scatter(
        x=df_stats['start'],
        y=df_stats['E_mean'],
        name='E_mean'
    ),
    go.Scatter(
        x=df_stats['start'],
        y=df_stats['acc_train'],
        name='acc_train'
    )
])

fig.show()

In [101]:
px.line(df_stats, x='start', y='acc_train')

In [102]:
neural_network.accuracy(df=data_test[['food', 'expected']])

0.56

In [98]:
EXPECTED = data_train['expected'].iloc[0]

own = neural_network.gradient(EXPECTED)
other = nd.Gradient(lambda x: neural_network.E(EXPECTED, x))(neural_network.parameters)

px.scatter(zip(own, other))

# Linear regression

In [10]:
test = list(range(10))
test[-3:]

[7, 8, 9]

In [209]:
nn.cross_entropy(predicted=np.array([0.775, 0.126, 0.039, 0.070]), expected=np.array([1, 0, 0, 0]))

0.25489224962879004